<a href="https://colab.research.google.com/github/Esaipriya/AISECT-ML/blob/main/RFC_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libaries
import pandas as pd
import numpy as np

In [ ]:
# load dataset
dataset=pd.read_csv("/content/Social_Network_Ads.csv")

In [ ]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [ ]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [ ]:
#one hot encoding
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [ ]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [ ]:
#assign  input output variables
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [ ]:
#standardscaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
#import algorithm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# passing parameters
from sklearn.model_selection import GridSearchCV, cross_val_predict
param_grid= {
    'n_estimators':[100],
    'criterion':["gini",'entropy','log_loss'],
    'max_depth':[None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True]
    }

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None],
                         'max_features': ['sqrt', 'log2', None],
                         'max_leaf_nodes': [None],
                         'min_impurity_decrease': [0.0],
                         'min_samples_leaf': [1], 'min_samples_split': [2],
                         'min_weight_fraction_leaf': [0.0],
                         'n_estimators': [100]},
             scoring='f1_weighted', verbose=3)

In [ ]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [ ]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [ ]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100}
The f1_macro: 0.8444991648183138
The confusion Matrix:
 [[228  29]
 [ 33 110]]
The report:
               precision    recall  f1-score   support

           0       0.87      0.89      0.88       257
           1       0.79      0.77      0.78       143

    accuracy                           0.84       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.84      0.84      0.84       400



In [ ]:
# Save Best Model
import pickle
filename = "RandomForest_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [ ]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_impurity_decrease,...,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.296858,0.018609,0.018816,0.000640,True,gini,None,sqrt,None,0.0,...,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.783072,0.925000,0.872729,0.840551,0.814270,0.847124,0.048878,4
1,0.296037,0.012047,0.019288,0.001674,True,gini,None,log2,None,0.0,...,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.767183,0.937229,0.872729,0.840551,0.839760,0.851491,0.055077,2
2,0.313175,0.012516,0.018068,0.000341,True,gini,None,None,None,0.0,...,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.750847,0.937229,0.859227,0.803611,0.730275,0.816238,0.075249,9
3,0.289757,0.001634,0.018511,0.000185,True,entropy,None,sqrt,None,0.0,...,100,"{'bootstrap': True, 'criterion': 'entropy', 'm...",0.767183,0.937229,0.884822,0.840551,0.851777,0.856313,0.055809,1
4,0.297946,0.012634,0.022303,0.005144,True,entropy,None,log2,None,0.0,...,100,"{'bootstrap': True, 'criterion': 'entropy', 'm...",0.783072,0.937229,0.871261,0.840551,0.790456,0.844514,0.056612,5


In [ ]:
import numpy as np
import pickle
import pandas as pd  # Import pandas

# Load the saved model from the specified file
loaded_model = pickle.load(open('/content/RandomForest_CV_best_model.sav', 'rb'))

# Get user input for age, estimated salary, and gender
age = int(input("Enter Age: "))
estimated_salary = int(input("Enter Estimated Salary: "))
gender = input("Enter Gender (Male/Female): ").lower()

# Convert gender input to numerical format (1 for Male, 0 for Female)
gender_male = 1 if gender == 'male' else 0

# Prepare the user input data as a pandas DataFrame with the correct feature names
# This matches the format the scaler was fitted on.
user_input_df = pd.DataFrame([[age, estimated_salary, gender_male]], columns=['Age', 'EstimatedSalary', 'Gender_Male'])


# Scale the user input using the same scaler 'sc' that was fitted on the training data (X)
# Note: It's crucial to use the same scaler fitted on the training data to scale new data
scaled_input = sc.transform(user_input_df)

# Make a prediction using the loaded model and the scaled user input
prediction = loaded_model.predict(scaled_input)

# Output the prediction result
if prediction[0] == 1:
    print("Purchased")
else:
    print("Not Purchased")

Enter Age: 19
Enter Estimated Salary: 19000
Enter Gender (Male/Female): male
Not Purchased
